In [1]:
import tensorflow as tf
import keras
from fastdtw import fastdtw
import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display
import pandas as pd
from tensorflow.keras.callbacks import ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
def dtw(df_in,df_out):
    in_data = np.zeros((25,0))
    out_data = np.zeros((25,0))
    distance, path = fastdtw(df_in.T,df_out.T)
    for i in range(1,len(path)):
        index_in, index_out = path[i]
        a = df_in[:,index_in:index_in+1]
        b = df_out[:,index_out:index_out+1]
        in_data = np.append(in_data, a, axis = 1)
        out_data = np.append(out_data, b, axis = 1)
    return (out_data, in_data)

In [3]:
def load_data(name_indata,name_outdata):
    in_data = np.zeros((25,0))
    out_data = np.zeros((25,0))
    for i in range(1,540):
        df_in = pd.read_csv('cmu_us_'+ name_indata +'_arctic/wav/mfcc/arctic_b00' + str(i) + '.csv',header=None,index_col=None)
        df_in = df_in.values
        df_out = pd.read_csv('cmu_us_'+ name_outdata +'_arctic/wav/mfcc/arctic_b00' + str(i) + '.csv',header=None,index_col=None)
        df_out = df_out.values
        df_in,df_out = dtw(df_in,df_out)
        in_data = np.append(in_data, df_in, axis=1)
        out_data = np.append(out_data, df_out, axis=1)
    for i  in range(51,590):
        df_in = pd.read_csv('cmu_us_'+ name_indata +'_arctic/wav/mfcc/arctic_a00' + str(i) + '.csv',header=None,index_col=None)
        df_in = df_in.values
        df_out = pd.read_csv('cmu_us_'+ name_outdata +'_arctic/wav/mfcc/arctic_a00' + str(i) + '.csv',header=None,index_col=None)
        df_out = df_out.values
        df_in,df_out = dtw(df_in,df_out)
        in_data = np.append(in_data, df_in, axis=1)
        out_data = np.append(out_data, df_out, axis=1)
    return (in_data, out_data)

In [4]:
def load_testset(name_indata,name_outdata):
    in_data = np.zeros((25,0))
    out_data = np.zeros((25,0))
    for i in range(1,50):
        df_in = pd.read_csv('cmu_us_'+ name_indata +'_arctic/wavtest/mfcc/arctic_a00' + str(i) + '.csv',header=None,index_col=None)
        df_in = df_in.values
        df_out = pd.read_csv('cmu_us_'+ name_outdata +'_arctic/wavtest/mfcc/arctic_a00' + str(i) + '.csv',header=None,index_col=None)
        df_out = df_out.values
        in_data = np.append(in_data, df_in, axis=1)
        out_data = np.append(out_data, df_out, axis=1)
    return (in_data,out_data)

In [5]:
#train and test set for different networks

X_Train_12,Y_Train_12 = load_data('awb','bdl')
X_Test_12,Y_Test_12 = load_testset('awb','bdl')

X_Train_13,Y_Train_13 = load_data('awb','ksp')
X_Test_13,Y_Test_13 = load_testset('awb','ksp')

X_Train_23,Y_Train_23 = load_data('bdl','ksp')
X_Test_23,Y_Test_23 = load_testset('bdl','ksp')

/home/kushal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  after removing the cwd from sys.path.


NameError: name 'indata' is not defined

In [ ]:
from keras.models import Sequential
from keras.layers import Dropout, Dense
from sklearn.model_selection import train_test_split
##remember to add dropout and batchnorm

In [ ]:
model = Sequential()

model.add(Dense(100,input_shape=(25,None),use_bias=False))
model.add(BatchNormalization()) #not quite sure on batch Norm params
model.add(Activation('tanh'))
model.add(Dropout(0.2))

model.add(Dense(100),use_bias=False)
model.add(BatchNormalization())
model.add(Activation('tanh'))
model.add(Dropout(0.2))

model.add(Dense(25),activation = 'tanh',use_bias=False)  ##output layer, does it need an activation?
model.compile(loss='mean_sqaured_error',optimizers='Adam',metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,patience=5, min_lr=0.001,verbose = 1)
model.fit(X_Train_12,Y_Train_12,batch_size=16*1120,epochs=5,verbose=1,validation_split=0.0,shuffle=True,callbacks=[reduce_lr])
#batch_size is natch_size*no.of frames per sample

In [ ]:
score = model.evaluate(X_Test_12,Y_Test_12,batch_size=8*1120,verbose=1)